In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchvision.transforms import ToTensor

In [3]:
flood_data = pd.read_csv("/content/datasets/kerala.csv")
flood_data['FLOODS'].replace(['YES','NO'],[1,0],inplace=True)
flood_data


,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL RAINFALL,FLOODS
0,KERALA,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4,3248.6,1
1,KERALA,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5,3326.6,1
2,KERALA,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0,3271.2,1
3,KERALA,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3,3129.7,1
4,KERALA,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2,2741.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,KERALA,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2,3046.4,1
114,KERALA,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4,2600.6,0
115,KERALA,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6,2176.6,0
116,KERALA,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1,2117.1,0


,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5
2,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3
4,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2
114,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4
115,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6
116,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1


In [ ]:
y = flood_data

In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
class FloodDataset(Dataset):
    def __init__(self, data_file, transform=None):
        df = pd.read_csv(data_file)

        # Split dataframe into data and labels
        self.data = df.drop(columns=['floods'])
        self.labels = df['floods']

    def __len__(self):
        # Return the size of the dataset
        return len(self.data)

    def __getitem__(self, index):
      item = self.data.iloc[index]
      label = self.labels[index]
      # Convert data and label to PyTorch tensors
      data_tensor = torch.tensor(item.values, dtype=torch.float32)
      label_tensor = torch.tensor(label, dtype=torch.long)
      # Apply transformations if available
      if self.transform:
        data_tensor = self.transform(data_tensor)

        return data_tensor, label_tensor



In [19]:
x = flood_data.iloc[:,1:14]
x

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5
2,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3
4,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2
114,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4
115,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6
116,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1


In [22]:
y = flood_data.iloc[:,-1]
y

0      1
1      1
2      1
3      1
4      0
      ..
113    1
114    0
115    0
116    0
117    1
Name: FLOODS, Length: 118, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x, test_size= 0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size= 0.2, random_state=42)
x_train

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
44,1945,4.4,9.9,12.7,104.1,53.4,549.8,704.0,351.8,110.9,253.8,259.5,18.0
12,1913,3.1,5.2,20.7,75.7,198.8,541.7,763.2,247.2,176.9,422.5,109.9,45.8
15,1916,0.0,7.8,22.0,82.4,199.0,920.2,513.9,396.9,339.3,320.7,134.3,8.9
70,1971,31.6,18.5,20.0,113.0,317.5,889.6,648.6,385.2,331.2,220.9,38.3,62.3
107,2008,0.8,30.3,217.2,108.4,81.2,469.9,505.1,349.0,347.0,343.4,55.4,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2007,0.5,5.6,7.3,138.5,192.7,705.9,966.3,489.6,526.7,357.2,87.4,11.9
14,1915,16.9,23.5,42.7,106.0,154.5,696.1,775.6,298.8,396.6,196.6,302.5,14.9
92,1993,0.0,17.8,20.1,66.5,159.0,657.1,776.1,301.9,88.0,431.2,153.8,46.2
51,1952,5.2,48.2,20.8,112.2,214.6,576.7,430.0,413.6,57.4,339.6,49.6,66.9


In [25]:
import torchvision.transforms as transforms
from torchvision import datasets

training_x = torch.tensor(x_train.values, dtype=torch.float32)
testing_x = torch.tensor(x_test.values, dtype=torch.float32)
training_y = torch.tensor(y_train.values, dtype=torch.float32)
testing_y = torch.tensor(y_test.values, dtype=torch.float32)

# train_data =FloodDataset(training_data,transform= transforms.ToTensor())

tensor([[1.9450e+03, 4.4000e+00, 9.9000e+00,  ..., 2.5380e+02, 2.5950e+02,
         1.8000e+01],
        [1.9130e+03, 3.1000e+00, 5.2000e+00,  ..., 4.2250e+02, 1.0990e+02,
         4.5800e+01],
        [1.9160e+03, 0.0000e+00, 7.8000e+00,  ..., 3.2070e+02, 1.3430e+02,
         8.9000e+00],
        ...,
        [1.9930e+03, 0.0000e+00, 1.7800e+01,  ..., 4.3120e+02, 1.5380e+02,
         4.6200e+01],
        [1.9520e+03, 5.2000e+00, 4.8200e+01,  ..., 3.3960e+02, 4.9600e+01,
         6.6900e+01],
        [2.0030e+03, 7.0000e-01, 5.0900e+01,  ..., 4.0700e+02, 7.6400e+01,
         9.7000e+00]])

In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils import data
#dataset chunk to use for each iteration
batchsize = 32

train_dataloader = DataLoader(train_data, batch_size= batchsize)
test_dataloader = DataLoader(test_data, batch_size = batchsize)

In [ ]:
#check the device used for training the model
import torch
device =(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

In [ ]:
import torch.nn as nn
class Distra(nn.Module):
  def _init_(self, input_size, hidden_size, num_layers, output_size):
    super (Distra, self). init ()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn. LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn. Linear (hidden_size, output_size)

    def forward (self, x):
      h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      c0= torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      out, = self.lstm(x, (h0, c0))
      out = self.fc (out[:, -1, ])
      return out
model= Distra().to_device()

In [ ]:

loss_fn = nn.CrossEntropyLoss()
#optimizer to adjust the weights and baises of the nueral network e.g
#stochastic gradient(SGD), Adam etc.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
#lr is the learning rate(alpha)


In [ ]:
#training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        #Zero_grad  sens the gradient to zero after every iteration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 80
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
#save model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
#load model
model = Distra()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
#making predictions
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  predict = model(x)